In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.plotly as py
import plotly
import statsmodels.api as sm
from scipy.stats import skewnorm as skn
from scipy.stats import norm
import statsmodels.api as sm
import plotly.graph_objs as go

C:\Users\xexx\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



The model includes both an offensive and defensive universe:
- Offensive: US equities (represented by SPY), international equities (EFA), emerging market equities (EEM) and US aggregate bonds (AGG). More on the seemingly odd inclusion of AGG as an offensive asset in a moment.
- Defensive: US corporate bonds (LQD), US intermediate-term Treasuries (IEF) and US short-term Treasuries (SHY).
- For all assets, at the close on the last trading day of the month, calculate a “momentum score” based on month-end data as follows:
$$12(\frac{p_0}{p_1} – 1) + 4(\frac{p_0}{p_3} – 1) + 2(\frac{p_0}{p_6} – 1) + (\frac{p_0}{p_{12}} – 1)$$
 - Where p0 = the asset price at today’s close, p1 = the asset price at the close of the previous month, etc.
 - Note how this approach overweights more recent months. Doing the math, the most recent 1-month change (p0/p1 – 1) determines 40% of the momentum score, while the most distant month (p11/p12 – 1) determines just ~2%.

- If all four of the offensive assets exhibit positive momentum scores, select the offensive asset with the highest score and allocate 100% of the portfolio to that asset at the close. Note the use of both absolute and relative momentum here, an idea popularized by Gary Antonacci as “Dual Momentum”. Why is that important? Historically, absolute momentum has done well minimizing losses, while relative momentum has helped in generating outsized returns.
- If any of the four offensive assets exhibit negative momentum scores, select the defensive asset (LQD, IEF or SHY) with the highest score (regardless of whether the score is > 0) and allocate 100% of the portfolio to that asset at the close. As we do throughout this site, trades in SHY are assumed to be placed in cash, as it’s more relevant to today’s market given SHY’s low yields coupled with the impact of transaction costs and how frequently this strategy trades.
- Hold the position until the final trading day of the following month.

In [2]:
# Use six to import urllib so it is working for Python2/3
from six.moves import urllib
# If you don't want to use six, please comment out the line above
# and use the line below instead (for Python3 only).
#import urllib.request, urllib.parse, urllib.error

import time

'''
Starting on May 2017, Yahoo financial has terminated its service on
the well used EOD data download without warning. This is confirmed
by Yahoo employee in forum posts.
Yahoo financial EOD data, however, still works on Yahoo financial pages.
These download links uses a "crumb" for authentication with a cookie "B".
This code is provided to obtain such matching cookie and crumb.
'''

# Build the cookie handler
cookier = urllib.request.HTTPCookieProcessor()
opener = urllib.request.build_opener(cookier)
urllib.request.install_opener(opener)

# Cookie and corresponding crumb
_cookie = None
_crumb = None

_headers={'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11'}

def get_cookie_crumb(ticker):
    '''
    This function perform a query and extract the matching cookie and crumb.
    '''

    # Perform a Yahoo financial lookup on SP500
    req = urllib.request.Request('https://finance.yahoo.com/quote/' + ticker, headers=_headers)
    f = urllib.request.urlopen(req)
    alines = f.read().decode('utf-8')

    # Extract the crumb from the response
    global _crumb
    cs = alines.find('CrumbStore')
    cr = alines.find('crumb', cs + 10)
    cl = alines.find(':', cr + 5)
    q1 = alines.find('"', cl + 1)
    q2 = alines.find('"', q1 + 1)
    crumb = alines[q1 + 1:q2]
    _crumb = crumb

    # Extract the cookie from cookiejar
    global cookier, _cookie
    for c in cookier.cookiejar:
        if c.domain != '.yahoo.com':
            continue
        if c.name != 'B':
            continue
    _cookie = c.value

    # Print the cookie and crumb
    # print('Cookie:', _cookie)
    # print('Crumb:', _crumb)
    return _crumb


def yahoo_historical(ticker = 'SPY', start_date = dt.datetime(2016,1,1)):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    start_date_unix = int(time.mktime(start_date.timetuple()))
    
    # Getting cookie crumb for yahoo finance query
    get_cookie_crumb(ticker)
    
    if ticker == 'VVIX':
        site = 'https://query1.finance.yahoo.com/v7/finance/download/%5EVVIX?period1={0}&period2='.format(start_date_unix) + str(int(time.time())) + '&interval=1d&events=history&crumb=' + get_cookie_crumb('^VVIX').replace('\\','')
    elif ticker == 'SPX':
        site = 'https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?period1={0}&period2='.format(start_date_unix) + str(int(time.time())) + '&interval=1d&events=history&crumb=' + get_cookie_crumb('^GSPC').replace('\\','')
    else:
        site = 'https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2='.format(ticker,start_date_unix) + str(int(time.time())) + '&interval=1d&events=history&crumb=' + get_cookie_crumb(ticker).replace('\\','')
    
    df = pd.read_csv(site, index_col = 0)
    return df

In [ ]:
ticker_list = ['AGG','EFA,'EEM'']